# Inteligêncial articial (2020/01)
### Trabalho em Grupo do Grau A

*Graph coloring problem (GCP)*

Integrantes:
*   Anderson
*   Felipe Silva
*   Gabriel Castro
*   Lucas Oliveira

Prof. Gabriel de Oliveira Ramos

---

# Definições do problema

O problema de coloração de grafos é definido por um conjuto de n nós.  O grafo resultante representa um mapa, onde os nós representam porções territoriais de países e as arestas representam a adjacência entre os territórios (territórios conectados são considerados vizinhos).

O objetivo deste *notebook* é criar um algoritmo que definirá uma cor para cada nó de modo que nós adjacentes (vizinhança) não possuam a mesma cor. E ao mesmo tempo, deve-se minimizar a quantidade de cores utilizadas.

**As intâncias disponíveis são:**
*   anna (cloração completa com 11 cores)
*   games120 (coloração completa com 9 cores)
*   myciel6 (coloração completa com 7 cores)

**Input de dados**

Os territórios são definidos através de um arquivo carregado, sendo ele formatado da seguinte maneira:
*   1º: tamanho do problema definido na primeira linha, onde:
    *  segundo item da linha define a quantidade de nós
    *  terceiro item da linha define a quantidade de arestas
*   2º: demais linhas representam as arestas do grafo, para isso:
    *  representamos a conexão entre um país e outro através de uma tupla

Para exemplo, demonstramos abaixo o formato do arquivo, com uma breve explicação de cada linha:

```
    10,19          --> 10: nós | 19: arestas
    br,ve          --> brasil - venezuela
    br,co          --> brasil - colômbia
    br,pe          --> brasil - perú
    ...
```
---


# Inicializações necessárias

In [0]:
import io
import pandas as pd
import random
import bisect
from google.colab import files

In [0]:
uploaded_countries = files.upload()

Saving conencted_countries.txt to conencted_countries (1).txt


Para iniciar, vamos usar a função `read_csv` da biblioteca pandas. Ela permite fazermos a leitura do arquivo carregado anteriormente, acessando assim os elementos necessários. 

Note que deixamos o arquivo sem cabeçalho, porém seria possível a utilização de um dentro do arquivo. Mas por escolha, foi definido ter somente os dados.

E já que não inserimos o nome das colunas no arquivo, o próprio DataFrame define como padrão o nome delas, enumerando de 0 até n quantidade de colunas - 1. Isso é uma coisa ruim para acessos posteriores, portanto, usamos a função `rename` para alterá-las.

In [0]:
df = pd.read_csv(io.BytesIO(uploaded_countries['conencted_countries.txt']), header=None)
df.rename(columns={
    0: 'x_country',
    1: 'y_country'
}, inplace=True)

Com os dados já armazenados no DataFrame, passamos a separá-los em duas estruturas de dados:
*   Tupla para a definição do problema (quantidade de nós e arestas);
*   Dicionário com listas combinando países conectados através das arestas.

Também foi definido um valor para cada país, através de um dicionário. Isso ajuda a identificar as arestas carregadas do arquivo.

Um exemplo de conexões pode ser visto abaixo:

<img src="https://raw.githubusercontent.com/gabcastro/Unisinos-IA-Exercicios/master/Trabalho_GA/schema_graph_7_colors.jpeg" alt="Exemplo de conexões" height="480">

In [0]:
problem_length = (df.head(1)['x_country'], df.head(1)['y_country'])     # tuple with len of nodes and edges

countries = {
    0: 'BR',
    1: 'UY',
    2: 'PY',
    3: 'AR',
    4: 'BO', 
    5: 'CH', 
    6: 'PE',
    7: 'CO', 
    8: 'EC',
    9: 'VE'
}

connected_countries = {}

for index, row in df.tail(len(df)-1).iterrows():
    str_countries = countries.get(row['x_country']) + '-' + countries.get(row['y_country'])
    connected_countries[str_countries] = [row['x_country'], row['y_country']]

# Algoritmo de busca para solução do problema

Para o problema de coloração de grafos, foi escolhido usar o algoritmo genético pois permite uma busca mais variada, diferente do Hill Climbing que após achar o primeiro ótimo local encerra a solução do problema.

Os AGs começam com um conjunto de *k* estados gerados aleatoriamente, chamado **população**. Cada estado, ou **indivíduo**, é representado como uma cadeia sobre um alfabeto finito. 

A produção de uma nova geração passa por algumas etapas, onde seguem uma sequência de passos:

*   **Função de adaptação (fitness)**: Retorna valores mais altos para estados melhores. A probabilidade de um indivíduo ser escolhido para reprodução é diretamente proporcional à sua pontuação de adaptação.
*   **Seleção**: Dois pares escolhidos aleatoriamente são selecionados para reprodução, de acordo com as probabilidades da função fitness.
*   **Cruzamento (*Crossover*)**: Pares são cruzados, a escolha é realizada ao acaso em um ponto de cruzamento dentre as posições na cadeia. Os próprios descendentes são criados por cruzamento das cadeias pais no ponto de crossover.
Ainda pode acontecer de dois estados pais serem bastante diferentes, fazendo com que a operação de cruzamento produza um estado que está longe do estado de qualquer pai.
*   **Mutação**: Cada posição está sujeita à mutação aleatória com uma pequena probabilidade independente.

---


Classe para calcular a aptidão de um indivíduo como o número de pares de territórios vizinhos que possuem cores diferentes.

In [0]:
class EvaluateGC:
    """Evaluation class.

    Since that a solution needs to be evaluated with respect to the problem instance 
    in consideration, we created this class to store the problem instance and to 
    allow the evaluation to be performed without having the problem instance at hand.
    """
    
    # during initialization, store the problem instance
    def __init__(self, problem_instance):
        self.problem_instance = problem_instance
    
    # compute the value of the received solution
    def __call__(self, solution):
        return sum(solution[n1] != solution[n2] for (n1, n2) in self.problem_instance.values())

In [0]:
class GeneticAlgorithm:

    def genetic_algorithm(self, population, fn_fitness, gene_pool, fn_thres=None, ngen=1000, pmut=0.1):
        """ Population creation pipeline
            
            Keywords arguments:
                population  -- initial population
                fn_fitness  -- evaluation function (EvaluateGC)
                gene_pool   -- set of possible values for each position of the solution
                fn_thres    -- stop criterion for solution quality (default None)
                ngem        -- maximum number of generations (default 1000)
                pmut        -- mutation probability (default 0.1)
        """
    
		# for each generation
		for i in range(ngen):

			# create a new population
			new_population = []

			# repeat to create len(population) individuals
			for i in range(len(population)):
			
				# select the parents
				p1, p2 = self.select(2, population, fn_fitness)

				# recombine the parents, thus producing the child
				child = self.recombine(p1, p2)

				# mutate the child
				child = self.mutate(child, gene_pool, pmut)

				# add the child to the new population
				new_population.append(child)

			# move to the new population
			population = new_population

			# check if one of the individuals achieved a fitness of fn_thres; if so, return it
			fittest_individual = self.fitness_threshold(fn_fitness,fn_thres,  population)
			if fittest_individual:
				return fittest_individual

		# return the individual with highest fitness
		return max(population, key=fn_fitness)
  

    def fitness_threshold(fn_fitness, fn_thres, population):
        """Get the best individual of the received population and return it if its 
        fitness is higher than the specified threshold fn_thres
        """

        if not fn_thres:
            return None

        fittest_individual = max(population, key=fn_fitness)
        if fn_fitness(fittest_individual) >= fn_thres:
            return fittest_individual

        return None

    
    def select(r, population, fn_fitness):
        """Implements the genetic selection operator.
        Genetic operator for selection of individuals; 
        This function implements roulette wheel selection, where individuals with 
        higher fitness are selected with higher probability.
        """

        fitnesses = map(fn_fitness, population)
        sampler = weighted_sampler(population, fitnesses)
        return [sampler() for i in range(r)]

    
    def weighted_sampler(seq, weights):
        """Return a single sample from sequence. 
        The probability of a sample being returned is proportional to its weight
        """

        totals = []
        for w in weights:
            totals.append(w + totals[-1] if totals else w)
        return lambda: seq[bisect.bisect(totals, random.uniform(0, totals[-1]))]

    
    def recombine(x, y):
        """Genetic operator for recombination (crossover) of individuals.

        This function implements single-point crossover, where the resulting individual
        carries a portion [0,c] from parent x and a portion [c,n] from parent y, with 
        c selected at random.
        """

        n = len(x)
        c = random.randrange(0, n)
        return x[:c] + y[c:]


    def mutate(x, gene_pool, pmut):
        """Genetic operator for mutation. 
        
        This function implements uniform mutation, where a single element of the 
        individual is selected at random and its value is changed by a randomly chosen 
        value (out of the possible values in gene_pool).
        """
        
        # if random >= pmut, then no mutation is performed
        if random.uniform(0, 1) >= pmut:
            return x

        n = len(x)
        g = len(gene_pool)
        c = random.randrange(0, n) # gene to be mutated
        r = random.randrange(0, g) # new value of the selected gene

        new_gene = gene_pool[r]
        return x[:c] + [new_gene] + x[c+1:]


    def init_population(pop_number, gene_pool, state_length):
        """Generate init population. 
        
        Will create a random population.

        Keywords arguments:
            pop_number      -- len of population
            gene_pool       -- possible values to each element
            state_length    -- number of genes for each individual
        """

        g = len(gene_pool)
        population = []
        for i in range(pop_number):
            # each individual is represented as an array with size state_length, 
            # where each position contains a value from gene_pool selected at random
            new_individual = [gene_pool[random.randrange(0, g)] for j in range(state_length)]
            population.append(new_individual)

        return population

Agora que a classe do algoritmo genético está criado, podemos configurar alguns parâmetros necessários e executá-la para ver o primeiro resultado.

In [0]:
# set of possible colours (pink, orange, green, gray, purple, yellow, blue)
possible_values = ['P', 'O', 'GN', 'GY', 'P', 'Y', 'B'] 

# length of an individual (one position per territory)
individual_length = problem_length[0]

# population size
population_size = 8

gen_alg = GeneticAlgorithm()

# initial population
population = gen_alg.init_population(population_size, possible_values, individual_length)

# evaluation class
fn_fitness = EvaluateGC(connected_countries)

# run the algoritm
solution = gen_alg.genetic_algorithm(population, fn_fitness, gene_pool=possible_values, fn_thres=10)

# print the results
print('Resulting solution: %s' % solution)
print('Value of resulting solution: %d' % fn_fitness(solution))